#

In [2]:
import ee
import pandas as pd
import geopandas as gpd
import geemap
import os
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import mapping

#ee.Authenticate()
ee.Initialize()

In [6]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [7]:
os.getcwd()

'/home/datascience/herbivore_nutrient_interactions'

In [12]:

# Path to your shapefile
shapefile_path = "data/raw/survey_data/australian_plague_locut_distribution/Australia_Locust_Distribution.shp"

# Read the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Convert the GeoPandas GeoDataFrame to an Earth Engine FeatureCollection
apl_map = geemap.geopandas_to_ee(gdf)

# Example of using it for further analysis in Google Earth Engine
Map = geemap.Map()
Map.centerObject(ee_fc)
Map.addLayer(ee_fc, {}, "Shapefile Layer")
Map

In [56]:
# Australia soil grid

# Load the Australian Soil Grid image collection.
soil_collection = ee.ImageCollection("CSIRO/SLGA")

nitrogen = soil_collection.filter(ee.Filter.inList('system:index', ['NTO'])).select(['NTO_000_005_EV','NTO_005_015_EV']).mean()

avg_nitrogen = nitrogen.expression(
    '(b1 + b2) / 2', {
        'b1': nitrogen.select('NTO_000_005_EV'),
        'b2': nitrogen.select('NTO_005_015_EV')
    }
).rename('avg_nto_band').clip(apl_map)

phosphorus = soil_collection.filter(ee.Filter.inList('system:index', ['PTO'])).select(['PTO_000_005_EV','PTO_005_015_EV']).mean()

avg_phosphorus = phosphorus.expression(
    '(b1 + b2) / 2', {
        'b1': phosphorus.select('PTO_000_005_EV'),
        'b2': phosphorus.select('PTO_005_015_EV')
    }
).rename('avg_pto_band').clip(apl_map)


avg_phosphorus = avg_phosphorus.updateMask(avg_phosphorus.clip(apl_map))
avg_nitrogen = avg_nitrogen.updateMask(avg_nitrogen.clip(apl_map))

In [57]:
# Example of using it for further analysis in Google Earth Engine
Map = geemap.Map()
Map.centerObject(avg_nitrogen)
Map.addLayer(avg_nitrogen, {}, "average phosphorus")
Map

In [59]:

# Export the clipped raster directly to a local file as a GeoTIFF
geemap.ee_export_image(
    avg_nitrogen, 
    filename = "data/raw/survey_data/apl_nitrogen_raster.tif",
    region = apl_map.geometry(), 
    scale = 2000
)

print("Nitrogen Raster file saved")

# Export the clipped raster directly to a local file as a GeoTIFF
geemap.ee_export_image(
    avg_phosphorus, 
    filename="data/raw/survey_data/apl_phosphorus_raster.tif",
    region=apl_map.geometry(), 
    scale = 2000
)

print("Phophorus Raster file saved")

Generating URL ...
Please wait ...
Data downloaded to /home/datascience/herbivore_nutrient_interactions/data/raw/survey_data/apl_nitrogen_raster.tif
Nitrogen Raster file saved
Generating URL ...
Please wait ...
Data downloaded to /home/datascience/herbivore_nutrient_interactions/data/raw/survey_data/apl_phosphorus_raster.tif
Phophorus Raster file saved